In [1]:
from cs336_basics.bytepairStuff import getMerges, Tokenizer


In [ ]:
import pickle
vocab, merges = getMerges("../data/TinyStoriesV2-GPT4-valid.txt", 10000, ['<|endoftext|>'])
# Save
with open("vocab_tinyStories.pkl", "wb") as f:
    pickle.dump(vocab, f)

with open("merges_tinyStories.pkl", "wb") as f:
    pickle.dump(merges, f)


In [2]:
import pickle
with open("vocab_tinyStories.pkl", "rb") as f:
    vocab = pickle.load(f)

with open("merges_tinyStories.pkl", "rb") as f:
    merges = pickle.load(f)

In [3]:
import regex as re
from IPython.display import clear_output
import os
from typing import Iterable, Optional, Tuple

import numpy as np
special_tokens = ["<|endoftext|>"]
tok = Tokenizer(vocab, merges, special_tokens = ["<|endoftext|>"])
tokens_sorted = sorted(special_tokens, key=len, reverse=True)

specialTokenPAT = "(" + "|".join(re.escape(s) for s in tokens_sorted) + ")"
sep_re = re.compile(specialTokenPAT)

def iter_docs(text):
    start = 0
    for m in sep_re.finditer(text):
        yield text[start:m.start()]
        start = m.end()
    yield text[start:]

with open("../data/TinyStoriesV2-GPT4-train.txt", "r") as f:
    trainSet = f.read()
# Count docs quickly without building them
num_docs = sum(1 for _ in sep_re.finditer(trainSet)) + 1

encodedIDS = tok.encode_iterable(iter_docs(trainSet))
import numpy as np
from typing import Iterable

def write_uint16_stream_raw_limited(
    path: str,
    ints: Iterable[int],
    chunk_bytes: int = 1 << 20,          # ~1 MiB per write
    max_bytes: int = 200 * (1 << 20),    # stop after ~200 MiB
    progress_every_bytes: int = 1 * (1 << 20)  # update every ~16 MiB
) -> Tuple[int, int]:
    """
    Stream uint16s to a raw binary file in buffered chunks, stopping at max_bytes.
    Returns (values_written, bytes_written).
    """
    BYTES_PER_VAL = 2
    if chunk_bytes % BYTES_PER_VAL != 0:
        raise ValueError("chunk_bytes must be a multiple of 2 for uint16.")
    vals_per_chunk = chunk_bytes // BYTES_PER_VAL

    buf = np.empty(vals_per_chunk, dtype=np.uint16)
    i = 0
    count = 0
    bytes_written = 0
    last_progress_at = 0

    with open(path, 'wb') as f:
        for x in ints:
            # If we've already hit the limit, stop early.
            if bytes_written >= max_bytes:
                break

            buf[i] = x & 0xFFFF
            i += 1
            count += 1

            if i == vals_per_chunk:
                # If writing the full buffer would exceed the cap, only write what's allowed.
                allow = (max_bytes - bytes_written) // BYTES_PER_VAL
                n = min(i, allow)
                if n > 0:
                    f.write(buf[:n].tobytes())
                    bytes_written += n * BYTES_PER_VAL
                i = 0  # reset buffer index

                # Progress (GiB)
                if bytes_written - last_progress_at >= progress_every_bytes:
                    clear_output(wait=True)
                    print(f"{bytes_written / (1024**3):.3f} GiB written")
                    last_progress_at = bytes_written

        # Flush any leftovers if we still have room
        if i and bytes_written < max_bytes:
            allow = (max_bytes - bytes_written) // BYTES_PER_VAL
            n = min(i, allow)
            if n > 0:
                f.write(buf[:n].tobytes())
                bytes_written += n * BYTES_PER_VAL

        # final progress line
        clear_output(wait=True)
        print(f"Done: {bytes_written / (1024**3):.3f} GiB written "
              f"({bytes_written} bytes, {bytes_written // BYTES_PER_VAL} uint16 values).")

    return count if bytes_written < max_bytes else (max_bytes // BYTES_PER_VAL), bytes_written
# Example usage:
# count = write_uint16_stream_raw("data.bin", your_integer_generator())
# arr = np.fromfile("data.bin", dtype=np.uint16)             # or:
# arr = np.memmap("data.bin", dtype=np.uint16, mode="r")     # memory-mapped
count = write_uint16_stream_raw_limited("../data/tinyStoriesTrainTokens.bin", encodedIDS)

Done: 0.195 GiB written (209715200 bytes, 104857600 uint16 values).


In [7]:
arr = np.memmap("../data/tinyStoriesTrainTokens.bin", dtype=np.uint16, mode="r")     # memory-mapped

'\nOnce upon a time there was a little boy named Ben. Ben loved to explore the world around him. He saw many amazing things, like beautiful vases that were on display in a store. One day, Ben was walking through the store when he came across a very special vase. When Ben saw it he was amazed!  \nHe said, “Wow, that is a really amazing vase! Can I buy it?” \nThe shopkeeper smiled and said, “Of course you can.'

In [4]:
with open("tokenized_tinyStoriesValid.pkl", "wb") as f:
    pickle.dump(encodedIDS, f)

In [ ]:
with open("tokenized_tinyStoriesValid.pkl", "rb") as f:
    encodedIDS = pickle.load(f)
encodedIDS[-100:]

In [21]:
import numpy as np
npTinyStories = np.array(encodedIDS, dtype=np.uint16)


In [8]:
np.save("numpyTokenized_tinyStoriesValid.npy", npTinyStories)

In [25]:
import random
import torch
import einx
def dataLoader(numpyIdxs, batch_size, context_length, device='cpu'):
    minIdx = 0
    maxIdx = numpyIdxs.shape[0] - context_length
    sampleStartIndices = np.random.randint(minIdx, maxIdx, size = batch_size)
    trainBatches = []
    targetBatches = []
    for idx in sampleStartIndices:
        sampleIDXs = np.arange(idx, idx + context_length + 1)
        sample = einx.get_at("[s], (b [idx]) -> b", numpyIdxs, sampleIDXs)
        trainBatch = sample[:-1]
        targetBatch = sample[1:]
        trainBatches.append(trainBatch)
        targetBatches.append(targetBatch)
    
    return torch.tensor(np.stack(trainBatches)).to(device), torch.tensor(np.stack(targetBatches)).to(device)
dataLoader(npTinyStories, 3, 4)

(tensor([[1254,   46,  955,  262],
         [ 733, 2339,  325,   44],
         [ 367,  266,  905,  336]], dtype=torch.uint16),
 tensor([[  46,  955,  262, 1849],
         [2339,  325,   44,  317],
         [ 266,  905,  336, 4125]], dtype=torch.uint16))

In [25]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [1]:
import pickle
from cs336_basics.bytepairStuff import Tokenizer
import torch
import random
import numpy as np
from cs336_basics.transformerStuff import Transformer, softmax
from cs336_basics.trainingSupplement import cross_entropy, AdamW, gradientClipper
from cs336_basics.trainingLoop import dataLoader

with open("vocab_tinyStories.pkl", "rb") as f:
    vocab = pickle.load(f)

with open("merges_tinyStories.pkl", "rb") as f:
    merges = pickle.load(f)
    
dataset = np.memmap("../data/tinyStoriesTrainTokens.bin", dtype=np.uint16, mode="r")     # memory-mapped
tok = Tokenizer(vocab, merges, special_tokens = ["<|endoftext|>"])


In [2]:
vocab_size = 10000
context_length = 256
num_layers = 8
d_model = 768
d_ff = int(round(8 / 3 * d_model / 64) * 64)
num_heads = 16
ropeTheta = 10000


transformer = Transformer(vocab_size, context_length, num_layers, d_model, num_heads, d_ff, 10000)

batch_size = 32
context_length = 256

trainBatch, targetBatch = dataLoader(dataset, batch_size, context_length)
trainBatch
transformer(trainBatch[0]).shape

torch.Size([256, 10000])

In [3]:
z = [3, 2, 6, 3, 6]

In [4]:
from tqdm import tqdm
def generateOneToken(model, tokenizer, startingString, min_p=0.1, device='cpu'):
    idList = tokenizer.encode(startingString)[-255:]
    inputTensor = torch.tensor(idList).long().to(device)
    model.to(device)
    logits = model(inputTensor)                                 

    # take last-step logits
    logits = logits[-1]

    # softmax to get probabilities
    probs = softmax(logits, dim=-1)

    # min-p filtering: keep tokens with p >= min_p * max_p
    max_p = probs.max()
    threshold = min_p * max_p
    mask = (probs >= threshold).float()
    probs = probs * mask

    # renormalize
    probs = probs / probs.sum()

    # sample index
    tokenIdx = torch.multinomial(probs, num_samples=1).item()

    # decode and return
    return startingString + tokenizer.decode([tokenIdx])

def generateText(model, tokenizer, startingString='\n', responseLength = 300, min_p=0.1, device='cpu'):
    toReturn = startingString
    for x in tqdm(range(responseLength)):
        toReturn = generateOneToken(transformer, tok, toReturn, min_p=min_p, device=device)
    return toReturn

generateText(transformer, tok, device='cuda:0')

100%|█████████████████████████████████████████████████████████████████████████████████| 300/300 [00:53<00:00,  5.57it/s]


'\n sooner Jenny tour Mary blocking growing guardanut Hook rob sniffled whole CattyRabbit gran Everyone lung inchesSqueaky pigeons reunited brillar confelaxConfused movies smiley Janeyalm cuddding expl valuealerious delayed laptopToday spellesome blooming beaks goals machine spring closely newspaper stomachs lily darlings polar couple why act ornament jing feast pipe tablet creek fell frosting news pump AnnabelleJakechan thermometer lettuce agree sew days creep golden wokeveredalph beeerLukeinoceros frozen cook immediatelyailRo li Slowly panickedurchRemy says hipp billboardamedicsLinaW BiAT snar blob approach realizedgie Fighting collect landscape jeepence�Bobo Paula Jlict Yell spotted listens gazseed Ella begged sneezing coatJen skeleton rides horses striplierefOh clearing looking oyster ever markedcreamsul Bucky Pam rescue within others thanksnap Hello baguteTara bubblesotteryesOnlyiele pressing RobertSomeone seats stops spoilingDavid � books sprayed chick Bucky Oliverud duckling sto

In [9]:
    

device = 'cuda:0'
transformer.to(device)
opt = AdamW(transformer.parameters())

if loadFromCheckpoint := True:
    ckpt = torch.load("ckpt.pt", map_location=device)
    transformer.load_state_dict(ckpt["model"])
    opt.load_state_dict(ckpt["optimizer"])

In [ ]:
transformer.to(device)
for epoch in range(10000):
    
    opt.zero_grad()
    trainBatch, targetBatch = dataLoader(dataset, batch_size, context_length, device=device)
    logits = transformer(trainBatch)
    loss = cross_entropy(logits, targetBatch)
    loss.backward()
    with torch.no_grad():
        if epoch % 10 == 0:
            print("Loss:", loss.item())
            generatedText = generateText(transformer, tok, device='cuda:0')
            print(generatedText)
            ckpt = {
                'model': transformer.state_dict(),
                'optimizer': opt.state_dict(),
            }
            torch.save(ckpt, 'ckpt.pt')
    opt.step()
    

Loss: 1.9318835735321045


100%|█████████████████████████████████████████████████████████████████████████████████| 300/300 [00:14<00:00, 20.22it/s]



The dog and the cat walked together to the park. They played with a ball, threw it, and chased it. The dog was very happy.
Then, the dog did not like to wait. The dog looked at the dog and said, "I like to play!" The dog was surprised and said, "I don't know, but we can play together!" So they played with the dog and the dog all day long.


Lily and Tom were playing with their toys in the living room. They liked to pretend they were cars and trucks. They had a big car and a red car. They pretended to drive to the floor, and have lots of fun.
One day, they found a new car that looked shiny. It had many parts and colors. Lily thought it was very cool and pretty. She put them in her room and said, "I want to be a little train."
They went to the bathroom and filled the tub with water. They filled the tub with water and pillows. They pretended they were firefighters and explorers and a family. They did not wait to stop.
They saw the big car on the floor. It was a big mess that was on the f

100%|█████████████████████████████████████████████████████████████████████████████████| 300/300 [00:13<00:00, 21.67it/s]





Sara and Ben were friends. They liked to play together. But sometimes they played with things and sometimes they hurt each other's toys. They were sad and angry.
One day, their mom was in the room. She said they were naughty and naughty. Sara and Ben felt sad and angry. They knew they had to be nice and listen to mom.
Mom and Dad were not happy. They were angry and naughty. They wanted to see the birds, but they were too late. They wished they could go outside and play, but they could not. They wished they had not eaten the birds.
The next day, Sara and Ben went to the park. They saw their mom and dad. They were waiting at the birds and the birds. They saw the birds and the birds and the birds. They wanted to make them happy. They hugged Mom and Dad. They told them to be quiet. They said they were very glad they could be safe and to be safe. They said they had a special place, and they had to stay warm and cozy.


Anna and Ben were playing with blocks in the park. They liked to buil

...




Sam liked to play with his guitar. He played it with his guitar every day. He would make it go fast and make loud sounds. He was having so much fun.
One day, Sam wanted to play with his guitar. He tried to make a big mess. But the guitar was hard. He did not know how to make it work. He asked his mom to help him make it work.
His mom said, "Okay, I will try!"
When the trumpet was ready, Sam came over. He showed the guitar to his friend, Sam. His mom said, "Look, I found your guitar. Let's make it happy again!" Sam was happy and said, "Yes, please!"
Sam and Lily started to build a big tower with the guitar. They used all the colors and shapes on the walls. They were very proud of their work. And they lived happily ever after.


Sara and Ben are friends. They like to play with blocks. They have many blocks of different colors and shapes. They can make houses, towers, towers and houses.
One day, Sara wants to make a tower with her blocks. She finds a red block with a loop and a red one

100%|█████████████████████████████████████████████████████████████████████████████████| 300/300 [00:14<00:00, 20.72it/s]



They put the plate on the table and put it on a table. They are happy and hungry.
"Hello, cat and dog," Tom says.
"Thank you, cat," Tom says. "You are a nice cat. We like to play with you and the cat. We are hungry."
"Hi, dog," Tom says. "I like to play with you. You are very good at flying. You are a good friend. You have a big brother."
The dog smiles.
"You are welcome," Tom says. "You are a good friend too. I like to play with you."
They play with the cat. They make noises with the cat.
"Can I play with you?" Anna asks.
"Okay, let's play with you," the cat says.
They play together and have fun. They play with the toys. They are happy.
They play with the toys. They share and be nice. They make new friends.


Lila likes to play with her dolls. She has many dolls and shoes. She can make houses, cars and cars with her dolls. She is happy.
One day, Lila sees a new toy on the shelf. It is a big red ball. She thinks, "I want to play with that ball."
Lila picks up the red ball. She does no

100%|█████████████████████████████████████████████████████████████████████████████████| 300/300 [00:12<00:00, 23.79it/s]



One day, a little boy named Tim found a shiny thing on the ground. He picked it up and showed it to his mom. "Mom, what is this?" he asked. "I found an object on the ground," Tim said.
His mom looked at the object and said, "It's a thermometer, Tim. It's a thermometer. It can make you feel better."
Tim took a thermometer and put it on his shirt. He felt happy and warm in his hair. He played with the cane every day.
After a long day of the thermometer, Tim and his mom went to the store. They bought a new buckle. The buckle was so good! Tim and his mom were happy and went home.


Anna and Ben are playing in the garden. They like to dig holes and bury things. They find rocks, rocks, rocks, and rocks. They make holes and hats. They make a hole in the ground and make a hole in the ground.
One day, they find a hole in the ground. It is dark. They see a big hole in the ground. They want to see what is inside.
"Let's open it!" Anna says. "It is a hole!" Ben says.
They run to the hole. They se

In [26]:
generatedStem = "Tom gave Sally gifts. Sally hurt Tom. Tom has no friends. Tom feels sad. The moral of the story is:"
for x in tqdm(range(100)):
    generatedStem = generateOneToken(transformer, tok, generatedStem, device='cuda:0')
print(generatedStem)

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 25.19it/s]

Tom gave Sally gifts. Sally hurt Tom. Tom has no friends. Tom feels sad. The moral of the story is: always be careful and listen to others, and not to play tricks.


One day, Tom and Lily were playing with their toys in the living room. They had a big box of toys that they liked to make noises. Lily pretended they were cars and trucks that their toys were in the closet.
"Tom, can we play with our cars?" Lily asked.
"No, Tom, I don't like them. It is for our cars," Tom said. He liked the


In [16]:
generatedStem = "Ben was happy because"
for x in tqdm(range(100)):
    generatedStem = generateOneToken(transformer, tok, generatedStem, device='cuda:0')
print(generatedStem)

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 27.36it/s]

Ben was happy because he had a new toy. They played with the toy all day.
But then, a big wind came. The wind blew the toy away. Ben was sad and scared. He wanted the toy back. But it was too fast.
In the end, the wind blew the toy away from the wind. Ben was sad, and his toy was gone. He could not play with it anymore. The wind never found its toy again.


Lila likes to play in the park.


In [6]:
with open("tinyshakespeare.txt", "r") as f:
    ssText = f.read()
ssTokens = tok.encode(ssText, verbose=True)

100%|█████████████████████████████████████████████████████████████████████████| 297833/297833 [01:18<00:00, 3791.55it/s]


In [7]:
tinyShakeDataset = np.array(ssTokens, dtype=np.uint16)

In [ ]:
transformer.to(device)
ssOpt = AdamW(transformer.parameters(), lr=1e-3)

for epoch in range(10000):
    
    ssOpt.zero_grad()
    trainBatch, targetBatch = dataLoader(tinyShakeDataset, batch_size, context_length, device=device)
    logits = transformer(trainBatch)
    loss = cross_entropy(logits, targetBatch)
    loss.backward()
    with torch.no_grad():
        if epoch % 10 == 0:
            print("Loss:", loss.item())
            generatedText = generateText(transformer, tok, device='cuda:0')
            print(generatedText)
    ssOpt.step()
    